In [5]:

import numpy as np
import pandas as pd
from typing import List
# import matplotlib.pyplot as plt

from lmfit import Minimizer, minimize, Parameters, fit_report

from lib.transformations import (euler_matrix, 
                                 euler_from_matrix, 
                                 scale_from_matrix,
)
from lib.utils import convert_to_homogeneous, print_results
from lib.io import read_data_to_df


def compute_residuals(
    params: Parameters,
    x0: np.ndarray, 
    x1: np.ndarray, 
    weights: np.ndarray = None,
    prior_covariance_scale: float = None, 
) -> np.ndarray: 
    ''' 3D rototranslation with scale factor

    X1_ = T_ + m * R * X0_

    Inputs: 
    - x0 (np.ndarray): Points in the starting reference system
    - x1 (np.ndarray): Points in final reference system 
    - weights (np.ndarray, defult = None): weights (e.g., inverse of a-priori observation uncertainty)
    - prior_covariance_scale (float, default = None): A-priori sigma_0^2     
    
    Return: 
    - res (nx1 np.ndarray): Vector of the weighted residuals
        
    '''    
        
    # Get parameters  
    parvals = params.valuesdict()
    rx = parvals['rx']
    ry = parvals['ry']
    rz = parvals['rz']
    tx = parvals['tx']
    ty = parvals['ty']
    tz = parvals['tz']
    m = parvals['m']
    
    # Convert points to homogeneos coordinates and traspose np array to obtain a 4xn matrix
    x0 = convert_to_homogeneous(x0).T

    # Build 4x4 transformation matrix (T) in homogeneous coordinates
    T = np.identity(4)
    R = euler_matrix(rx, ry, rz)
    T[0:3, 0:3] = (m * np.identity(3)) @ R[:3,:3]
    T[0:3, 3:4] = np.array([tx, ty, tz]).reshape(3, 1) 
    
    # Apply transformation to x0 points
    x1_ = T @ x0
    x1_ = x1_[:3,:].T 

    # Compute residuals as differences between observed and estimated values, scaled by the a-priori observation uncertainties
    res = (x1 - x1_)
    
    # If weigthts are provided, scale residual
    if weights is not None:
        
        res = res * weights
    
    return res.flatten()
    

# 
# MAIN FCT 
# 

pt_loc = read_data_to_df('../data/rotra3d/lingua_loc.txt',
                         header=None,
                         col_names=['label', 'x', 'y', 'z'],
                         index_col=0
                         )
pt_utm = read_data_to_df('../data/rotra3d/lingua_utm.txt',
                         header=None,
                         col_names=['label', 'x', 'y', 'z'],
                         index_col=0
                         )

# Remove df rows
rows_to_drop = ['F1']
pt_loc = pt_loc.drop(rows_to_drop)
pt_utm = pt_utm.drop(rows_to_drop)


t_ini = np.array([16000., 90000., 1700.], dtype='float64')
rot_ini = np.array([0., 0., 0.], 'float64')
m_ini = float(1.0)

# Define Parameters to be optimized
params = Parameters()
params.add('rx', value=rot_ini[0], vary=False)
params.add('ry', value=rot_ini[1], vary=False)
params.add('rz', value=rot_ini[2], vary=True)
params.add('tx', value=t_ini[0], vary=True)
params.add('ty', value=t_ini[1], vary=True)
params.add('tz', value=t_ini[2], vary=True)
params.add('m',  value=m_ini, vary=True)

# Define Weights as the inverse of the a-priori standard deviation of each observation
# All the measurements are assumed as independent (Q diagonal)
# Weight matrix must have the same shape as the observation matrix (or vector) X0
uncertainty = np.concatenate((
    np.tile(2e-3, (4, 3)),
    np.tile(3e-2, (1, 3))
),
    axis=0,
)
weights = 1. / uncertainty

# A-priori Sigma_0²: scale of the covariance matrix
sigma0_2 = 1
    
# Run Optimization!
minimizer = Minimizer(
    compute_residuals,
    params,
    fcn_args=(
        pt_loc.to_numpy(),
        pt_utm.to_numpy(), 
    ),
    fcn_kws={
        'weights': weights,
        'prior_covariance_scale': sigma0_2,
    },
    scale_covar=False,
)
result = minimizer.minimize(method='leastsq')

   
# Print result
print_results(result, weights, sigma0_2)




-------------------------------
Optimization report
[[Fit Statistics]]
    # fitting method   = leastsq
    # function evals   = 31
    # data points      = 15
    # variables        = 5
    chi-square         = 64.8907776
    reduced chi-square = 6.48907776
    Akaike info crit   = 31.9698297
    Bayesian info crit = 35.5100807
[[Variables]]
    rx:  0 (fixed)
    ry:  0 (fixed)
    rz:  0.78559318 +/- 2.6591e-05 (0.00%) (init = 0)
    tx:  16614.8211 +/- 0.00492002 (0.00%) (init = 16000)
    ty:  90932.7197 +/- 0.00462566 (0.00%) (init = 90000)
    tz:  1767.50990 +/- 0.00225438 (0.00%) (init = 1700)
    m:   0.99947565 +/- 2.4915e-05 (0.00%) (init = 1)
[[Correlations]] (unreported correlations are < 0.100)
    C(rz, tx) = 0.979
    C(ty, m)  = -0.976
    C(tz, m)  = -0.896
    C(ty, tz) = 0.875
-------------------------------
Chi quadro test:
Degrees of freedom: 10
Chi2 empirical: 6.489
Chi2 limit: 18.307
Test passed
-------------------------------
Residuals
     X       Y      Z
  

In [4]:
# Remove df rows
print(pt_loc)
rows_to_drop = ['F1']
pt_loc = pt_loc.drop(rows_to_drop)
print(pt_loc)


              x         y          z
label                               
201    146.7654  100.0006   88.99196
300    167.4933  146.6870   66.73523
400     99.9995   99.9989  100.02010
500    109.3794  155.0236   68.68320
F1      49.6488  192.0875   71.74660
F1BIS   53.2401  163.0449   78.29600
              x         y          z
label                               
201    146.7654  100.0006   88.99196
300    167.4933  146.6870   66.73523
400     99.9995   99.9989  100.02010
500    109.3794  155.0236   68.68320
F1BIS   53.2401  163.0449   78.29600
